Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [2]:
# Import Statements
import num2words
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import seaborn as sns
import re
import gensim


%matplotlib inline

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
pd.options.display.max_colwidth = 750

In [4]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [5]:
train.head()

,id,description,ratingCategory
0,1321,"\r\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only) A$133",1
1,3861,"\r\nAn uncommon exclusive bottling of a 6 year old cask strength malt. Light gold in color, the nose is vegetal, more peat bog than peat smoke, with an undercurrent of pastry cream and rose. It’s an odd combination of aromas. The entry is flavorful and inviting with smoked pineapple, clove, and rose. Peak smoke arrives in full force in the mid-palate, which drops the sweet and becomes spicy. The finish is mostly smoke, but with a pleasant minty coolness. (Wyoming only)",0
2,655,"\r\nThis release is a port version of Amrut’s Intermediate Sherry — a sort of port pipe sandwich. The spirit is matured in both unused casks and bourbon casks, then spends a few months in port pipes, and then returns to bourbon casks. The result is a Pink Floyd show of a whisky: vibrant, colorful, complex, and nearly too much. A blackcurrant and wispy, smoky nose gives way to an intense and bittersweet mix of chili, blackcurrant, oak, damson, dark chocolate, and peat. Astounding.",1
3,555,"\r\nThis 41 year old single cask was aged in a sherry butt and interacted magnificently with that wood. The nose offers peeled red apples, sultanas, honey, fudge, milk chocolate, and American cream soda. Balanced and harmonious throughout, the palate yields remarkably fresh orchard fruits, pipe tobacco, black pepper, and ginger. Long in the finish, with cinnamon drying to dark chocolate, and slightly smoky, unobtrusively tannic oak. A 1970s classic!",1
4,1965,"\r\nQuite herbal on the nose, with aromas of dried tarragon, parsley, and dill, along with cherry, milk chocolate, orange blossom, vanilla bean, and sweet tea. The palate is slightly grain-forward, but is full and well balanced, with orange, dried flowers, chamomile tea, lemon rind, and ginger. The finish offers cinnamon and milk chocolate against a note of candied lemon. Youthful and barrel-influenced, but admirably executed.",1


In [6]:
target = 'ratingCategory'

X = train.drop(columns=target)
y = train[target]

In [7]:
def clean_data(text):
    non_alpha = '[^a-zA-Z0-9]'
    multi_white_spaces = "[ ]{2,}"
    
    text = re.sub(non_alpha, ' ', text)
    text = re.sub(multi_white_spaces, " ", text)
    text = re.sub(r"(\d+)", lambda x: num2words.num2words(int(x.group(0))), text)
    
    # apply case normalization 
    return text.lower().lstrip().rstrip()

In [8]:
X['main_x'] = X['description'].apply(clean_data)
X

,id,description,main_x
0,1321,"\r\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only) A$133","sometimes when whisky is batched a few leftover barrels are returned to the warehouse canadian club recently pulled and vatted several of these from the one thousand, nine hundred and seventys acetone granny smith apples and fresh cut white cedar showcase this long age complex and spicy yet reserved this dram is ripe with strawberries canned pears cloves pepper and faint flowers then slightly pulling oak tannins distinct elegant and remarkably vibrant this ancient canadian club is anything but tired australia only a one hundred and thirty-three"
1,3861,"\r\nAn uncommon exclusive bottling of a 6 year old cask strength malt. Light gold in color, the nose is vegetal, more peat bog than peat smoke, with an undercurrent of pastry cream and rose. It’s an odd combination of aromas. The entry is flavorful and inviting with smoked pineapple, clove, and rose. Peak smoke arrives in full force in the mid-palate, which drops the sweet and becomes spicy. The finish is mostly smoke, but with a pleasant minty coolness. (Wyoming only)",an uncommon exclusive bottling of a six year old cask strength malt light gold in color the nose is vegetal more peat bog than peat smoke with an undercurrent of pastry cream and rose it s an odd combination of aromas the entry is flavorful and inviting with smoked pineapple clove and rose peak smoke arrives in full force in the mid palate which drops the sweet and becomes spicy the finish is mostly smoke but with a pleasant minty coolness wyoming only
2,655,"\r\nThis release is a port version of Amrut’s Intermediate Sherry — a sort of port pipe sandwich. The spirit is matured in both unused casks and bourbon casks, then spends a few months in port pipes, and then returns to bourbon casks. The result is a Pink Floyd show of a whisky: vibrant, colorful, complex, and nearly too much. A blackcurrant and wispy, smoky nose gives way to an intense and bittersweet mix of chili, blackcurrant, oak, damson, dark chocolate, and peat. Astounding.",this release is a port version of amrut s intermediate sherry a sort of port pipe sandwich the spirit is matured in both unused casks and bourbon casks then spends a few months in port pipes and then returns to bourbon casks the result is a pink floyd show of a whisky vibrant colorful complex and nearly too much a blackcurrant and wispy smoky nose gives way to an intense and bittersweet mix of chili blackcurrant oak damson dark chocolate and peat astounding
3,555,"\r\nThis 41 year old single cask was aged in a sherry butt and interacted magnificently with that wood. The nose offers peeled red apples, sultanas, honey, fudge, milk chocolate, and American cream soda. Balanced and harmonious throughout, the palate yields remarkably fresh orchard fruits, pipe tobacco, black pepper, and ginger. Long in the finish, with cinnamon drying to dark chocolate, and slightly smoky, unobtrusively tannic oak. A 1970s classic!","this forty-one year old single cask was aged in a sherry butt and interacted magnificently with that wood the nose offers peeled red apples sultanas honey fudge milk chocolate and american cream soda balanced and harmonious throughout the palate yields remarkably fresh orchard fruits pipe tobacco black pepper and ginger long in the finish with cinnamon drying to dark chocolate and slightly smoky unobtrusively tannic oak a one thousand, nine hundred and seventys classic"
4,1965,"\r\nQuite herbal on the nose, with aromas of dried tarrag

In [9]:
print(y.shape)
print(X.shape)

(4087,)
(4087, 3)


### Define Pipeline Components

In [10]:
tfidf = TfidfVectorizer()
rfc = RandomForestClassifier()

pipe = Pipeline([("vect", tfidf), 
                 ("clf", rfc)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [11]:
# parameters = {
#     "vect__max_df":(1.0, .95, .90),
#     "vect__min_df":(1, .05, .10),
#     "vect__max_features":(10000, 5000, 1000),
#     "vect__stop_words":("english", None),
#     "clf__n_estimators":(350)),
#     "clf__max_depth":(None, 5, 10)
# }

# grid_search = GridSearchCV(pipe, 
#                            parameters, 
#                            n_jobs=-2, 
#                            cv=3, 
#                            verbose=1)

# grid_search.fit(X['main_x'], y)

In [12]:
# grid_search.best_score_
grid_search_score = 0.7416211755402644

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [13]:
# # Predictions on test sample
# pred = grid_search.predict(test['description'])

In [14]:
# submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
# submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [15]:
# # Make Sure the Category is an Integer
# submission.head()

In [16]:
subNumber = 0

In [17]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

# submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 80% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


In [18]:
X.head(1)

,id,description,main_x
0,1321,"\r\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only) A$133","sometimes when whisky is batched a few leftover barrels are returned to the warehouse canadian club recently pulled and vatted several of these from the one thousand, nine hundred and seventys acetone granny smith apples and fresh cut white cedar showcase this long age complex and spicy yet reserved this dram is ripe with strawberries canned pears cloves pepper and faint flowers then slightly pulling oak tannins distinct elegant and remarkably vibrant this ancient canadian club is anything but tired australia only a one hundred and thirty-three"


### Define Pipeline Components

In [19]:
svd = TruncatedSVD()
vect = TfidfVectorizer(stop_words="english")
clf = RandomForestClassifier()

lsi = Pipeline([("vect", tfidf),("svd", svd)])
pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [20]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': (0.75, 1.0),
    "clf__n_estimators":(330, 340, 350, 360, 370),
    "clf__max_depth":(None, 5, 10)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-2, verbose=1)
grid_search.fit(X['main_x'], y)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-2)]: Done 186 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-2)]: Done 436 tasks      | elapsed: 15.8min
[Parallel(n_jobs=-2)]: Done 450 out of 450 | elapsed: 16.4min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [37]:
grid_search.best_score_

0.731833621125652

### Make a Submission File

In [22]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [23]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [24]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [25]:
# Save your Submission File
# # Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [26]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [27]:
import spacy
import pandas as pd

# build a model that is trained on word vectors 

# load in pre-trained w2v model 
nlp = spacy.load("en_core_web_lg")

In [28]:
# Continue Word Embedding Work Here
def get_word_vectors(docs):
    """
    This serves as both our tokenizer and vectorizer.
    Returns a list of (averaged) word vectors, i.e. our doc-term matrix 
    """
    
    return [nlp(doc).vector  for doc in docs ]

In [29]:
# clean text data
train["clean_text"] = train.description.apply(clean_data)
test["clean_text"] = test.description.apply(clean_data)

In [30]:
# move clean text data to arrays
X_train = train["clean_text"].values
X_test = test["clean_text"].values

In [31]:
# create train/test vectors
X_train_vect = get_word_vectors(X_train)
X_test_vect = get_word_vectors(X_test)

In [32]:
# move training labels to array 
Y_train = train.ratingCategory.values

In [33]:
gbc = GradientBoostingClassifier()

random_search = RandomizedSearchCV(gbc, 
                                   param_dist,
                                   cv=5, 
                                   n_jobs=-2, 
                                   verbose=1
                                   )
random_search.fit(X_train_vect, Y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed: 26.0min
[Parallel(n_jobs=-2)]: Done  50 out of  50 | elapsed: 32.8min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                        criterion='friedman_mse',
                                                        init=None,
                                                        learning_rate=0.1,
                                                        loss='deviance',
                                                        max_depth=3,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                    

In [38]:
random_search.best_score_

0.7448004357285435

### Make a Submission File

In [42]:
# Predictions on test sample
pred = random_search.predict(X_test_vect)

In [43]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [44]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [45]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?